In [2]:
import pandas as pd
import numpy as np

In [3]:
docuConfirma = pd.read_excel("Covid Chile V2 EXPERIMENTO.xlsx",  sheet_name="COVID_CL_CONFIRMA")
docuMuertes = pd.read_excel("Covid Chile V2 EXPERIMENTO.xlsx",  sheet_name="COVID_CL_MUERTE")
docuFinal = pd.read_excel("Covid Chile V2 EXPERIMENTO.xlsx",  sheet_name="FINAL")

In [4]:
docuConfirma.dropna(axis = 0, how = 'all', inplace = True)
if(docuConfirma.columns[0] == 'Unnamed: 0'):
    docuConfirma.columns = docuConfirma.iloc[0]
    docuConfirma = docuConfirma.iloc[1:,].reindex()
    
docuMuertes.dropna(axis = 0, how = 'all', inplace = True)
if(docuMuertes.columns[0] == 'Unnamed: 0'):
    docuMuertes.columns = docuMuertes.iloc[0]
    docuMuertes = docuMuertes.iloc[1:,].reindex()

In [5]:
dfFecha = pd.DataFrame({"Fecha" : docuConfirma["Fecha"]})
res = dfFecha.groupby(['Fecha']).sum().reset_index()

df = pd.DataFrame(columns = docuFinal.columns)
df3 = pd.DataFrame({"Region" : docuConfirma["Region"], "Comuna" : docuConfirma["Comuna"]})

res2 = df3.groupby(["Region","Comuna"]).sum().reset_index()


In [29]:
lista = []
for Comuna in res2["Comuna"]:
    for Fecha in res["Fecha"]:
        lista.append(Fecha)
        
res3 = pd.concat([res2]*res["Fecha"].count()).sort_index()
res3['Fecha'] = lista
res3['Casos Diarios'] = 0
res3['Fallecidos Diarios'] = 0

In [30]:
df4 = pd.DataFrame({"Region" : docuConfirma["Region"], "Comuna" : docuConfirma["Comuna"], "Fecha" : docuConfirma["Fecha"], "Casos Diarios" : 0})

resConfirma4 = df4.groupby(["Region","Comuna","Fecha"]).count().reset_index()
resConfirma4.insert(4, "Fallecidos Diarios", 0, True)
resConfirma5 = pd.concat([ res3 ,  resConfirma4 ])
resConfirma6 = resConfirma5.groupby(['Region','Comuna','Fecha']).sum().reset_index()

In [31]:
df5 = pd.DataFrame({"Region" : docuMuertes["Region"], "Comuna" : docuMuertes["Comuna"], "Fecha" : docuMuertes["Fecha"], "Fallecidos Diarios" : 0})
resMuerte4 = df5.groupby(["Region","Comuna","Fecha"]).count().reset_index()
resMuerte4.insert(3, "Casos Diarios", 0, True) 
resTotal = pd.concat([ resConfirma6 ,  resMuerte4])
resTotal = resTotal.groupby(['Region','Comuna','Fecha']).sum().reset_index()

In [32]:
resTotal["Muertes"] = resTotal['Fallecidos Diarios']
for i in range(1, len(resTotal)):
    if(resTotal.loc[i, 'Comuna'] == resTotal.loc[i-1, 'Comuna']):
        resTotal.loc[i, 'Muertes'] = resTotal.loc[i, 'Muertes'] + resTotal.loc[i-1, 'Muertes']

In [33]:
resTotal

,Region,Comuna,Fecha,Casos Diarios,Fallecidos Diarios,Muertes
0,Antofagasta,Antofagasta,2020-03-03,0,0,0
1,Antofagasta,Antofagasta,2020-03-04,0,0,0
2,Antofagasta,Antofagasta,2020-03-05,0,0,0
3,Antofagasta,Antofagasta,2020-03-06,0,0,0
4,Antofagasta,Antofagasta,2020-03-07,0,0,0
...,...,...,...,...,...,...
17951,Ñuble,Ñiquén,2020-04-30,0,0,1
17952,Ñuble,Ñiquén,2020-05-01,0,0,1
17953,Ñuble,Ñiquén,2020-05-02,0,0,1
17954,Ñuble,Ñiquén,2020-05-03,0,0,1


In [34]:
resTotal.to_csv("ResTotal.csv", index=False)